<h1>Make a Network Graph File</h1>
This code will generate a file to upload into Gephi Lite so that you can visualise your network.

This 'Jupyter Lab' workspace is made up of blocks of code. To run each block click the 'play' button above.

In [ ]:
import io
import pandas as pd
from ipywidgets import FileUpload
import networkx as nx
import math
upload = FileUpload(accept='.xlsx', multiple=False)

<h3>Upload your excel file</h3> 

Run this next block of code then click upload to select your excel file. If done correctly, the "Upload(0)" button should change to "Upload(1)" to show that you have uploaded 1 file. 

In [ ]:
upload

<h3>Create your network file</h3>

Run the following block of code to generate a GEXF file for you network. If successful, you should see a 'done!' message appear below the code.

In [ ]:
def makeGraph(df, actors, graphName):

    try:
        df = df.drop(['total'])
        df = df.drop(columns=['total'])
    except:
        pass
    
    nodeDict = {}
    for k,v in actors.iterrows():
        # print(k)
        nodeDict['{k}_{g}'.format(k=k, g=graphName)] = {
            'name': k,
            'role': v['Role'],
            'weight': [],
        }
    
    edgeDict = {}
    cols = list(df)
    names = []
    for col in cols:
        try:
            col = float(col)
        except ValueError:
            if col != 'total':
                names.append(col)
   
    for k, v in df.iterrows():
        if k in names:
            n1 = '{k}_{g}'.format(k=k, g=graphName)
            
            for name in names:
                n2 = '{k}_{g}'.format(k=name, g=graphName)
                
                if n1 != n2:
                    edge = tuple(sorted([n1, n2]))
                    edgeWeight = v[name]
                          
                    if edge not in edgeDict.keys():
                        edgeDict[edge] = {
                            'weight': []
                        }

                    if math.isnan(edgeWeight) == True:
                        edgeWeight = 0
                    edgeDict[edge]['weight'].append(edgeWeight)   
                    nodeDict[n1]['weight'].append(edgeWeight)
                    nodeDict[n2]['weight'].append(edgeWeight)
 
    return nodeDict, edgeDict

data = upload.value[0]
dfActor = pd.read_excel(io.BytesIO(data['content']), sheet_name='Actors', header=0, index_col=0)

# make graph
G = nx.Graph()

for stage in ('P1', 'P2', 'P3'):
    print('{s} started ....'.format(s=stage))
    df = pd.read_excel(io.BytesIO(data['content']), sheet_name='Point{n}'.format(n=stage[-1]), header=0, index_col=0)
    nodeDict, edgeDict = makeGraph(df, dfActor, stage)
    
    for n in nodeDict.keys():
        try:
            weight = sum(nodeDict[n]['weight']) / len(nodeDict[n]['weight'])
        except:
            weight = 0
        name = nodeDict[n]['name']
        role = nodeDict[n]['role']
        G.add_node(n, weight=weight, name=name, role=role)

    for e in edgeDict.keys():
        try:
            weight = sum(edgeDict[e]['weight']) / len(edgeDict[e]['weight'])
        except:
            weight = 0
        if weight > 0:
            G.add_edge(e[0], e[1], weight=weight)

    # write file
    filename = data['name']
    filename = '{f}_{n}_GEXF.gexf'.format(f=filename.split('.')[0], n=stage)
    print('{f} created'.format(f=filename))
    nx.write_gexf(G, filename)

print('done!')

<h3>Download your GEXF file!</h3>

Finally, click the 'folder' symbol in the left hand panel to find your GEXF file. Right-click on the file and download to your computer!